In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Just_normal/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'GKV_data.csv'
GKV_data = pd.read_csv(path + filename)

In [ ]:
set_index(GKV_data)

In [ ]:
data = pd.DataFrame(columns=['garch', 'return'], index=GKV_data.index)

In [ ]:
data['garch'] = GKV_data["('GARCH', 'normal', 0)"].values

In [ ]:
data['return'] = GKV_data["log_returns"].values

In [ ]:
data['GKV'] = GKV_data["GKV"].values

In [ ]:
data

,garch,return,GKV
Date,,,
2000-02-02,0.800313,-0.027564,1.022722
2000-02-03,0.655577,0.893688,0.984551
2000-02-04,0.491617,-0.052641,0.871380
2000-02-07,0.408321,-0.025746,0.848724
2000-02-08,0.339040,0.976294,0.796995
...,...,...,...
2024-08-26,0.093281,-0.275848,0.259302
2024-08-27,-0.162765,0.111656,0.251908
2024-08-28,-0.126100,-0.507014,0.223501


# Split train, validation and test data

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [ ]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [ ]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## GKV

In [ ]:
daily_train, daily_val, daily_test = split_data(data, 'GKV')

In [ ]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [ ]:
GKV_X_train, GKV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
GKV_X_val, GKV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
GKV_X_test, GKV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [ ]:
# GKV_X_train = GKV_X_train.reshape(GKV_X_train.shape[0], win_sz, -1)
# GKV_X_val = GKV_X_val.reshape(GKV_X_val.shape[0], win_sz, -1)
# GKV_X_test = GKV_X_test.reshape(GKV_X_test.shape[0], win_sz, -1)

### LSTM

In [ ]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(GKV_X_train.shape[1], GKV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,037 (54.83 KB)

 Trainable params: 14,037 (54.83 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_history = lstm_garch_model.fit(GKV_X_train, GKV_y_train, epochs=80, batch_size=32, validation_data=(GKV_X_val, GKV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 0.3192 - val_loss: 0.1540
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0942 - val_loss: 0.1361
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0907 - val_loss: 0.1185
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0764 - val_loss: 0.1683
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0707 - val_loss: 0.2230
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0641 - val_loss: 0.2657
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0614 - val_loss: 0.1984
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0537 - val_loss: 0.1824
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0597 - val_loss: 0.1380
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0495 - val_loss: 0.2142
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0439 - val_loss: 0.2307
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Predictions
GKV_test_predictions = lstm_garch_model.predict(GKV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [ ]:
mse = mean_squared_error(GKV_y_test, GKV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(GKV_y_test, GKV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(GKV_y_test, GKV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 0.06794732309398906
RMSE: 0.26066707328312305
MAE: 0.2074455072770518
MAPE: 81.37906425426364%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(GKV_X_train.shape[1], GKV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │           7,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,741 (41.96 KB)

 Trainable params: 10,741 (41.96 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gru_history = gru_model.fit(GKV_X_train, GKV_y_train, epochs=80, batch_size=32, validation_data=(GKV_X_val, GKV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.2679 - val_loss: 0.1231
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.1073 - val_loss: 0.1918
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0898 - val_loss: 0.1448
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.0899 - val_loss: 0.2501
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.1021 - val_loss: 0.1334
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.1422 - val_loss: 0.1793
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0683 - val_loss: 0.2416
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0547 - val_loss: 0.2099
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0643 - val_loss: 0.1816
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0458 - val_loss: 0.1382
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0398 - val_loss: 0.1536
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [34]:
# Predictions
GKV_gru_predictions = gru_model.predict(GKV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [35]:
mse = mean_squared_error(GKV_y_test, GKV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(GKV_y_test, GKV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(GKV_y_test, GKV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.04685205796723683
RMSE: 0.2164533621065675
MAE: 0.1673027278212291
MAPE: 73.92079504086674%


### LSTM_GRU

In [36]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(GKV_X_train.shape[1], GKV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
lstm_gru_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,045 (50.96 KB)

 Trainable params: 13,045 (50.96 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
lstm_gru_history = lstm_garch_model.fit(GKV_X_train, GKV_y_train, epochs=80, batch_size=32, validation_data=(GKV_X_val, GKV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0152 - val_loss: 0.1487
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0168 - val_loss: 0.2397
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0152 - val_loss: 0.1928
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0137 - val_loss: 0.1827
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0169 - val_loss: 0.1880
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0192 - val_loss: 0.1921
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0145 - val_loss: 0.1978
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0207 - val_loss: 0.1599
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0144 - val_loss: 0.1925
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0143 - val_loss: 0.1630
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0167 - val_loss: 0.1926
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [39]:
# Predictions
GKV_lstm_gru_predictions = lstm_garch_model.predict(GKV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [40]:
mse = mean_squared_error(GKV_y_test, GKV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(GKV_y_test, GKV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(GKV_y_test, GKV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.049925524077126894
RMSE: 0.2234402024639409
MAE: 0.17554408291838014
MAPE: 85.50297140765421%
